In [12]:
!pip3 install transformers

You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
!pip3 install faiss-cpu

You should consider upgrading via the 'pip install --upgrade pip' command.


In [14]:
!pip3 install -U scikit-learn scipy matplotlib

Requirement already up-to-date: scikit-learn in /Users/acupps/Library/Python/3.8/lib/python/site-packages (1.3.2)
Requirement already up-to-date: scipy in /Users/acupps/Library/Python/3.8/lib/python/site-packages (1.10.1)
Requirement already up-to-date: matplotlib in /Users/acupps/Library/Python/3.8/lib/python/site-packages (3.7.5)
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
!pip3 install networkx==3.1
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
from transformers import AutoTokenizer, AutoModel
import torch

/Users/acupps/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [135]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5')

We'll need to load our player data from our scraped JSON file that stores MLB player data:

In [279]:
import json

json_file_path = 'myfirstproject/myfirstproject/json_data/mlb_data_with_urls.json'

# load the JSON data from file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# filter to include only dictionaries with non-empty 'biographical_information'
valid_entries = [entry for entry in data if 'biographical_information' in entry and entry['biographical_information'].strip()]

# create the sentences list from the filtered valid_entries list
sentences = [entry['biographical_information'] for entry in valid_entries]

Now we need to tokenize the above sentences and prepare them for input to the BERT model. We'll create a dictionary with two keys: `input_ids` and `attention_mask`. These will be populated with the tokenized versions of the sentences.

In [280]:
# initialize dictionary to store tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=512,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

The next section will prepare the tokenized sentences for processing with the BERT model by reformatting the lists of tensors into single tensors.

In [281]:
# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

Below, the BERT model will process the tokenized inputs:

In [282]:
def batchify(data, batch_size):
    """Yield consecutive batches of size `batch_size` from `data`."""
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

# Define your batch size
batch_size = 5  # Adjust this based on your system's capabilities

# Initialize lists to store batched outputs
batched_outputs = []

with torch.no_grad():
    # Process in batches
    for batch_ids, batch_mask in zip(batchify(tokens['input_ids'], batch_size), batchify(tokens['attention_mask'], batch_size)):
        # Run the model on each batch
        outputs = model(input_ids=batch_ids, attention_mask=batch_mask)
        batched_outputs.append(outputs)
        

print("done modeling")


done modeling


In [172]:
for outputs in batched_outputs:
    print(outputs.keys())
    

odict_keys(['last_hidden_state', 'pooler_output'])
odict_keys(['last_hidden_state', 'pooler_output'])
odict_keys(['last_hidden_state', 'pooler_output'])
odict_keys(['last_hidden_state', 'pooler_output'])


This will extract the embeddings from the `last_hidden_state` tensor from the model's outputs to the `embeddings` variable. `last_hidden_state` is a tensor containing the final layer's hidden states for each token in each input sequence.

In [286]:
all_embeddings = []
for output in batched_outputs:
    all_embeddings.append(output.last_hidden_state)

embeddings = torch.cat(all_embeddings, dim=0)
embeddings.shape

torch.Size([2578, 512, 384])

In [287]:
embeddings

tensor([[[-0.1933,  0.2922, -0.0316,  ..., -0.6406,  0.3504,  0.1789],
         [-0.2810,  0.1790,  0.3708,  ..., -0.6255,  0.7429,  0.6992],
         [-0.3746,  0.2180,  0.3342,  ..., -0.2500,  0.0590, -0.4258],
         ...,
         [-0.2928,  0.0356,  0.1131,  ..., -0.4120,  0.4960,  0.3349],
         [-0.0494,  0.1417,  0.2669,  ..., -0.5017,  0.6249,  0.3162],
         [-0.1933,  0.2922, -0.0317,  ..., -0.6406,  0.3504,  0.1789]],

        [[-0.3537, -0.0124,  0.1551,  ..., -0.4708,  0.2704,  0.8026],
         [-0.1047, -0.0197,  0.3054,  ..., -0.0522,  0.6789,  0.8809],
         [ 0.0369, -0.0584,  0.4165,  ..., -0.2867,  1.3656, -0.1929],
         ...,
         [-0.3535, -0.0120,  0.1553,  ..., -0.4710,  0.2703,  0.8028],
         [-0.3536, -0.0121,  0.1553,  ..., -0.4709,  0.2703,  0.8028],
         [-0.1191, -0.1704,  0.4413,  ..., -0.5985,  0.3026,  0.4541]],

        [[-0.3684,  0.2967, -0.3442,  ..., -0.6297,  0.0565,  0.5579],
         [-0.2648,  0.2442,  0.0237,  ..., -0

After we have produced our dense vectors embeddings, we need to perform a mean pooling operation to create a single vector encoding (the sentence embedding). To do this mean pooling operation, we will need to multiply each value in our embeddings tensor by its respective attention_mask value — so that we ignore non-real tokens.

In [292]:
# resize our attention_mask tensor:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([2578, 512])

In [294]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([2578, 512, 384])

Each vector above represents a single token attention mask - each token now has a vector of size 768 representing it's attention_mask status. Then we multiply the two tensors to apply the attention mask:

In [295]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

torch.Size([2578, 512, 384])

"Mean Pooling" starts

In [296]:
# Then we sum the remained of the embeddings along axis 1, because we want to reduce the 512 tokens to 1 dimension
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([2578, 384])

we want to count only those values that we want to give attention
then divide by the sum to get the mean

In [297]:
# clamp returns the same tensor with a range given, clamp is used to replace the zeros to a very minimal value
# to avoid divide by zero error
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([2578, 384])

Finally, we calculate the mean as the sum of the embedding activations summed divided by the number of values that should be given attention in each position `summed_mask`:

In [299]:
mean_pooled = summed / summed_mask

`mean_pooled` is the final "dense representation" of the sentences, note that mean_pooled contains all representations for all sentences together

In [300]:
mean_pooled

tensor([[-0.1845,  0.3163,  0.2080,  ..., -0.6100,  0.4914,  0.1752],
        [-0.2176,  0.0125,  0.3889,  ..., -0.4580,  0.4477,  0.6827],
        [-0.3200,  0.2935, -0.0938,  ..., -0.5611,  0.1503,  0.4513],
        ...,
        [-0.2383,  0.3124,  0.1519,  ..., -0.2037, -0.0816,  0.2589],
        [ 0.0193, -0.3073,  0.1475,  ..., -0.0456,  0.5093,  0.4060],
        [-0.1927, -0.2879,  0.3511,  ..., -0.4951,  0.6355,  0.1398]])

## Cosine Similarity

In [301]:
def convert_to_embedding(query):
    tokens = {'input_ids': [], 'attention_mask': []}
    new_tokens = tokenizer.encode_plus(query, max_length=512,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])
    tokens['input_ids'] = torch.stack(tokens['input_ids'])
    tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state
    attention_mask = tokens['attention_mask']
    mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    masked_embeddings = embeddings * mask
    summed = torch.sum(masked_embeddings, 1)
    summed_mask = torch.clamp(mask.sum(1), min=1e-9)
    mean_pooled = summed / summed_mask
    
    return mean_pooled[0] # assuming query is a single sentence
    

In [302]:
from sklearn.metrics.pairwise import cosine_similarity

In [303]:
query = "yelich"
query_embedding = convert_to_embedding(query)

In [304]:
mean_pooled.shape

torch.Size([2578, 384])

In [305]:
cos = torch.nn.CosineSimilarity()
sim = cos(query_embedding, mean_pooled)
sim

tensor([0.5504, 0.4686, 0.4785,  ..., 0.4047, 0.4304, 0.4325])

# FAISS

In [306]:
import faiss                   # make faiss available
index = faiss.IndexFlatIP(384) # build the index
print(index.is_trained)
index.add(mean_pooled)        # add vectors to the index
print(index.ntotal)

True
2578


In [307]:
mean_pooled.shape

torch.Size([2578, 384])

In [308]:
query_embedding.shape

torch.Size([384])

In [309]:
D, I = index.search(query_embedding[None, :], 1)

In [310]:
D

array([[46.861103]], dtype=float32)

In [311]:
I

array([[6]])

In [318]:
faiss.write_index(index,"player_bios_faiss.index")

In [319]:
index_loaded = faiss.read_index("player_bios_faiss.index")

In [320]:
num_results = 5
D, I = index_loaded.search(query_embedding[None, :], num_results)

In [321]:
D

array([[46.861103, 46.861103, 40.720448, 40.720448, 40.63079 ]],
      dtype=float32)

In [322]:
I

array([[   7,    6, 1922,  780, 1341]])

Let's print out the results of the highest scoring document as a result of our query:

In [323]:
for i in range(0, num_results):
    most_similar_document_index = I[0][i]  # access the top doc result at index
    doc = valid_entries[most_similar_document_index]["biographical_information"]
    print("doc " + str(most_similar_document_index) + ": " + doc)

doc 7: Christian Yelich made his major league debut in 2013.
 He hit .455 as a high school senior, with 14 doubles and 9 home runs in 77 at-bats; he also stole 27 bases. Baseball America named him a second-team high school All-American infielder, behind first-teamers Sean Coyle, Nick Delmonico, Manny Machado and Chris Hawkins. He was the first first baseman taken in the 2010 amateur draft. He went 23rd overall, to the Florida Marlins; the scout was Tim McDonnell. He was converted to the outfield after signing with the Marlins (for a $1.7 million bonus) and began his pro career in 2010. He showed he belonged right away, going 3 for 3 with a triple, two runs and two RBI in his pro debut. He saw limited action for the GCL Marlins (9 for 24, 2B, 3B, 2 BB, SB) and Greensboro Grasshoppers (8 for 23, 2 2B, BB) that summer.
 A teenager in a full-season league in 2011, he more than held his own for Greensboro, hitting .312/.388/.484 with 32 doubles, 15 home runs, 73 runs, 77 RBI and 32 steals i